In [18]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization, LSTM, GRU, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
df = pd.read_csv('train3.csv')

In [24]:
X = df['comment_text'].astype(str) 
y = df[df.columns[1:]].values

In [5]:
MAX_FEATURES = 20000
SEQUENCE_LENGTH = 300
BATCH_SIZE = 24
EPOCHS = 10

In [6]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=SEQUENCE_LENGTH,
                               output_mode='int')
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(15000)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [9]:
model = tf.keras.Sequential([
    Embedding(MAX_FEATURES + 1, 128),
    Bidirectional(LSTM(256, activation='tanh', return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(128, activation='tanh')),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(5, activation='sigmoid')
])

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(train, epochs=EPOCHS, validation_data=val, callbacks=[early_stopping])

Epoch 1/10
613/613 [==============================] - 110s 162ms/step - loss: 0.2441 - accuracy: 0.9713 - val_loss: 0.1605 - val_accuracy: 0.9919
Epoch 2/10
613/613 [==============================] - 96s 157ms/step - loss: 0.1406 - accuracy: 0.9650 - val_loss: 0.1148 - val_accuracy: 0.7202
Epoch 3/10
613/613 [==============================] - 99s 161ms/step - loss: 0.1153 - accuracy: 0.9319 - val_loss: 0.1087 - val_accuracy: 0.9921
Epoch 4/10
613/613 [==============================] - 97s 159ms/step - loss: 0.1059 - accuracy: 0.8666 - val_loss: 0.0930 - val_accuracy: 0.9931
Epoch 5/10
613/613 [==============================] - 97s 158ms/step - loss: 0.0936 - accuracy: 0.7917 - val_loss: 0.0886 - val_accuracy: 0.2807
Epoch 6/10
613/613 [==============================] - 97s 158ms/step - loss: 0.0858 - accuracy: 0.7468 - val_loss: 0.0766 - val_accuracy: 0.9867
Epoch 7/10
613/613 [==============================] - 96s 157ms/step - loss: 0.0756 - accuracy: 0.7237 - val_loss: 0.0683 - val_a

In [17]:
model.save('romanizedd.h5')

In [13]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [14]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 2s 2s/step


array([[1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [15]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [16]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 94ms/step


In [17]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.9391534328460693, Recall:0.8934563994407654, Accuracy:0.24137930572032928


In [43]:
import tensorflow as tf
import gradio as gr

In [44]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[1:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [45]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

c:\Users\santo\miniconda3\envs\tensor\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\santo\miniconda3\envs\tensor\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Users\santo\miniconda3\envs\tensor\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [46]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://98ec4e77247cfd10ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 70ms/step
